In [1]:
import numpy as np
from sklearn.datasets import load_iris

In [2]:
data = load_iris()

In [3]:
print data.DESCR

Iris Plants Database

Notes
-----
Data Set Characteristics:
    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20  0.76     0.9565  (high!)
    ============== ==== ==== ======= ===== ====================

    :Missing Attribute Values: None
  

sepal - чашелистник<br>
petal - лепесток

In [4]:
data.data.shape

(150, 4)

In [5]:
data.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [6]:
data.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [7]:
data.target_names

array(['setosa', 'versicolor', 'virginica'], 
      dtype='|S10')

In [8]:
setosa_data = data.data[:50, :]
versicolor_data = data.data[50:100, :]
virginica_data = data.data[100:, :]

In [9]:
np.random.shuffle(setosa_data)
np.random.shuffle(versicolor_data)
np.random.shuffle(virginica_data)

In [10]:
setosa_data_train = setosa_data[:40]
setosa_data_test = setosa_data[40:]

versicolor_data_train = versicolor_data[:40]
versicolor_data_test = versicolor_data[40:]

virginica_data_train = virginica_data[:40]
virginica_data_test = virginica_data[40:]

$$ρ(𝑢, 𝑥_𝑖 )=\sqrt{∑_{𝑗=1}^{𝑛}(|𝑢^𝑗−𝑥_𝑖^𝑗|^2)} $$

In [11]:
def distance(u, x):
    diff = u - x
    return np.dot(np.transpose(diff), diff) ** 0.5    

Написать функции всех метрических алгоритмов, которые были на слайдах (ближайшего соседа, k ближайших соседей, k взвешанных ближайших соседей, метод окна Парзена, метод потенциальных функций). Применить (обучить) все эти алгоритмы на train выборке. Построить матрицу неточностей, для каждого класса вывести точность и полноту (все это сделать на тестовой выборке). Не пользоваться стандартными функциями.

In [64]:
class Classifier(object):    
    def __init__(self, distance):
        self.distance = distance
        
    def fit(self, X, y, n_classes):
        self.X_train = X
        self.y_train = y
        self.n_classes = n_classes
        
    def countDistances(self, u):
        distances = []
        for i in range(len(self.X_train)):
            distances.append([self.distance(u, X_train[i]), i])
        return distances        

In [65]:
class WeightKnn(Classifier):
    def predictElement(self, u, distances): 
        classes = [0 for i in range(self.n_classes)]
        for s in sorted(distances)[0:self.k_neighbors]:
            classes[y_train[s[1]]] += self.weight(X_train[i], u)
        return  sorted(range(len(classes)), key=lambda i : classes[i], reverse=True)[0]
    
    def predict(self, X, k_neighbors, weight):
        self.k_neighbors = k_neighbors
        self.weight = weight
        predicted = []
        distances = self.countDistances(X)
        for x in X:
            predicted.append(self.predictElement(x, distances))
        return predicted   

In [66]:
class Knn(WeightKnn):
    def predict(self, X, k_neighbors):
        return WeightKnn.predict(self, X, k_neighbors, lambda x, y: 1)
    

In [54]:
class Parzen(WeightKnn):
    def predictParzenElement(self, u):
        distances = self.countDistances(u)
        classes = [0 for i in range(self.n_classes)]
        h = sorted(distances)[self.k_neighbors - 1]
        self.weight = lambda x, u: Kernel(self.distance(u, x) / h)
        return self.predictElement(distances) 
    
    def predict(self, X, k_neighbors, kernel):
        self.kernel = kernel
        self.k_neighbors = k_neighbors
        predicted = []
        for x in X:
            predicted.append(self.predictParzenElement(x))
        return predicted 
    
    def predictFixedWindow(self, X, k_neighbors, kernel, h):
        return self.predict(X, lambda x, u: kernel(self.distance(u, x) / h), k_neigbors)     
    


In [55]:
def Potential(WeightKnn):
    def predictPotentialElement(self, u):
        distances = self.countDistances(u)
        classes = [0 for i in range(self.n_classes)]
        h = sorted(distances)[self.k_neighbors - 1]
        self.weight = lambda x, u: Kernel(self.distance(u, x) / h)
        return self.predictElement(distances) 
    
    def predict(self, X, kernel, potentials, radii):
        self.kernel = kernel
        self.potentials = potentials
        self.radii = radii
        predicted = []
        for x in X:
            predicted.append(self.predictPotentialElement(x))
        return predicted
        

In [56]:
def kernel1(r):
    if r < 0 or r > 1:
        return 0
    return 0.75 * ((1 - r) ** 2)

In [57]:
X_train = []
X_train.extend(setosa_data_train)
X_train.extend(versicolor_data_train)
X_train.extend(virginica_data_train)

y_train = [0 for i in range(40)]
y_train.extend([1 for i in range(40)])
y_train.extend([2 for i in range(40)])

X_test = []
X_test.extend(setosa_data_test)
X_test.extend(versicolor_data_test)
X_test.extend(virginica_data_test)

y_test = [0 for i in range(10)]
y_test.extend([1 for i in range(10)])
y_test.extend([2 for i in range(10)])

n_classes = 3

In [67]:
knn = Knn(distance)
knn.fit(X_train, y_train, n_classes)

In [68]:
y_predicted = knn.predict(X_test, 1)
print y_predicted

/home/yulya/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in sqrt
  app.launch_new_instance()


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
def get_confusion_matrix(y_expected, y_predicted, k):
    matrix = []
    for i in range(k):
        matrix.append([0 for i in range(k)])
    for i in range(len(y_expected)):
            matrix[y_expected[i]][y_predicted[i]] += 1
    return matrix

In [ ]:
def get_precision(confusion_matrix, c):
    return 1.0 * confusion_matrix[c][c] / sum(confusion_matrix[c])

In [ ]:
def get_recall(confusion_matrix, c):
    summ = 0.0
    for i in range(len(confusion_matrix)):
        summ += confusion_matrix[i][c]
    return confusion_matrix[c][c] / summ

In [ ]:
matrix = get_confusion_matrix(y_predicted, y_test, 3)
print(matrix)

In [ ]:
precisions = []
recalls = []
for i in range(3):
    precisions.append(get_precision(matrix, i))
    recalls.append(get_recall(matrix, i))
print(precisions)
print(recalls)